In [1]:
using Test

In [2]:
using Distributions

In [3]:
using CSV, DataFrames

In [4]:
using HDF5

In [5]:
using Statistics
using StatsBase
using LinearAlgebra

In [6]:
using Plots

In [7]:
using Printf

In [8]:
include("../src/ATools.jl")

Main.ATools

# Tests
1. test ray tracing

### Tests ray-tracking

In [9]:
c = ATools.Cylinder(1.0, 0.0, 1.0) 

# check that point (1,0,0) verifies cylinder equation 
    

@test ATools.cylinder_equation(c, [1.0, 0.0, 0.0]) ≈ 0.0
@test ATools.cylinder_equation(c, [0.0, 1.0, 0.0]) ≈ 0.0
@test isapprox(ATools.cylinder_equation(c, [1.0/sqrt(2.0), 1.0/sqrt(2.0), 0.0]), 0.0, atol=1e-9)


Test Passed

In [10]:
nb = ATools.normal_to_barrel(c,  [1.0, 2.0, 3.0])
@test dot(nb,[0.0, 0.0, 1.0]) ≈ 0.0

Test Passed

In [11]:
ATools.cylinder_length(c)

1.0

In [12]:
@test ATools.cylinder_length(c) ≈ 1.0
    
@test ATools.perimeter(c) ≈ 2 * π 

@test ATools.area_barrel(c) ≈ 2 * π 
    
@test ATools.area_endcap(c) ≈ π 
    
@test ATools.area(c) ≈ 4 * π

@test ATools.volume(c) ≈ π 

Test Passed

In [30]:
function propagate(r::ATools.Ray, cy::ATools.Cylinder)
    
    # intersection roots 
    a = r.d[1]^2 + r.d[2]^2
    b = 2 * (r.e[1] * r.d[1] + r.e[2] * r.d[2])
    c = r.e[1]^2 + r.e[2]^2 - cy.r^2

    println("r = ", r)
    println("cyl = ", cy)
    println("a =",a, " b = ", b, " c = ", c)

    # xray(t) propagates ray along t 
    xray = ATools.propagate_ray(r)

    # If no real roots return nothing 
    arg = b^2 - 4*a*c

    println("arg =",arg)
    if arg < 0
        return nothing 
    end

    # two possible roots 
    t1 = (-b + sqrt(arg))/(2*a)
    t2 = (-b - sqrt(arg))/(2*a)

    println("t1 =",t1, " t2 = ", t2)

    # Z position of the two roots 
    z1 = r.e[3] + t1 * r.d[3]
    z2 = r.e[3] + t2 * r.d[3]

    println("z1 =",z1, " z2 = ", z2)

    # belong-to-cylinder condition 
    zc1 = cy.zmin < z1 < cy.zmax
    zc2 = cy.zmin < z2 < cy.zmax

    println("zc1 =",zc1, " zc2 = ", zc2)

    if zc1 == false && zc2 == false
        return nothing
    elseif zc1 == true && zc2 == false
        return t1, xray(t1)
    elseif zc1 == false && zc2 == true
        return t2, xray(t2)
    else
        if t1 > 0 && t2 < 0
            return t1, xray(t1)
        elseif t1 < 0 && t2 > 0
            return t2, xray(t2)
        elseif t1 > 0 && t2 > 0

            if t1 < t2 
                return t1, xray(t1)
            else
                return t2, xray(t2)
            end
        else
            return nothing
        end
    end
end


propagate (generic function with 1 method)

In [37]:
r1 = ATools.Ray([0.0,0.0,0.5],[0.0,1.0,0.0])
t, p = propagate(r1, c)

r = Main.ATools.Ray([0.0, 0.0, 0.5], [0.0, 1.0, 0.0], [0.0, -0.8944271909999159, 0.4472135954999579])
cyl = Main.ATools.Cylinder(1.0, 0.0, 1.0)
a =1.0 b = 0.0 c = -1.0
arg =4.0
t1 =1.0 t2 = -1.0
z1 =0.5 z2 = 0.5
zc1 =true zc2 = true


(1.0, [0.0, 1.0, 0.5])

In [41]:
p == [0.0, 1.0, 0.5]

true

In [32]:
r = ATools.Ray([0.0,0.0,0.25],[0.0,1.0,0.0])

Main.ATools.Ray([0.0, 0.0, 0.25], [0.0, 1.0, 0.0], [0.0, -0.9701425001453319, 0.24253562503633297])

In [33]:
propagate(r, c)

r = Main.ATools.Ray([0.0, 0.0, 0.25], [0.0, 1.0, 0.0], [0.0, -0.9701425001453319, 0.24253562503633297])
cyl = Main.ATools.Cylinder(1.0, 0.0, 1.0)
a =1.0 b = 0.0 c = -1.0
arg =4.0
t1 =1.0 t2 = -1.0
z1 =0.25 z2 = 0.25
zc1 =true zc2 = true


(1.0, [0.0, 1.0, 0.25])

In [42]:
r = ATools.Ray([0.0,0.0,0.25],[sin(π/4.0),cos(π/4.0),0.0])

Main.ATools.Ray([0.0, 0.0, 0.25], [0.7071067811865475, 0.7071067811865476, 0.0], [-0.6859943405700353, -0.6859943405700354, 0.24253562503633297])

In [43]:
propagate(r, c)

r = Main.ATools.Ray([0.0, 0.0, 0.25], [0.7071067811865475, 0.7071067811865476, 0.0], [-0.6859943405700353, -0.6859943405700354, 0.24253562503633297])
cyl = Main.ATools.Cylinder(1.0, 0.0, 1.0)
a =1.0 b = 0.0 c = -1.0
arg =4.0
t1 =1.0 t2 = -1.0
z1 =0.25 z2 = 0.25
zc1 =true zc2 = true


(1.0, [0.7071067811865475, 0.7071067811865476, 0.25])

In [75]:
c = ATools.Cylinder(5.0, -100.0, 100.0) 
r = ATools.Ray([0.0,0.0,0.0],[sin(π/8.0),0.0,0.0])

Main.ATools.Ray([0.0, 0.0, 0.0], [0.3826834323650898, 0.0, 0.0], [-1.0, 0.0, 0.0])

In [76]:
propagate(r, c)

r = Main.ATools.Ray([0.0, 0.0, 0.0], [0.3826834323650898, 0.0, 0.0], [-1.0, 0.0, 0.0])
cyl = Main.ATools.Cylinder(5.0, -100.0, 100.0)
a =0.14644660940672624 b = 0.0 c = -25.0
arg =14.644660940672624
t1 =13.065629648763766 t2 = -13.065629648763766
z1 =0.0 z2 = 0.0
zc1 =true zc2 = true


(13.065629648763766, [5.0, 0.0, 0.0])